In [23]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense
import numpy as np
import sentencepiece as spm

batch_size = 64
epochs = 100
hidden_dims = 256
num_samples = 45093

data_file = "jpn.txt"
input_tokens = []
target_tokens = []
start_token = "<"
end_token = ">"

with open(data_file, "r", encoding="utf-8") as f:
    lines_list = f.read().split("\n")

tokenizer = spm.SentencePieceProcessor()
tokenizer.Load("spm.model")

for line in lines_list:
    #for the last black data, we need to skip
    if line == "":
        break
    input_text, target_text = line.split("\t")
    target_text = start_token + target_text + end_token
    tokenized_input = tokenizer.EncodeAsPieces(input_text)
    tokenized_target = tokenizer.EncodeAsPieces(target_text)
    
    inp_int_tokens = []
    targ_int_tokens = []
    for inp in tokenized_input:
        inp_int_tokens.append(tokenizer.piece_to_id(inp))
    for targ in tokenized_target:
        if targ == "▁":
            continue
        targ_int_tokens.append(tokenizer.piece_to_id(targ))
    
    input_tokens.append(inp_int_tokens)
    target_tokens.append(targ_int_tokens)
    
    
len(input_tokens), len(target_tokens)

(45093, 45093)

In [24]:
max_enc_seq = max([len(i) for i in input_tokens])
max_dec_seq = max([len(i) for i in target_tokens])

max_enc_seq, max_dec_seq

(128, 63)

In [25]:
vocab_size = 8000
hidden_dims = 256

##Network Architecture
enc_inputs = Input(shape=(None, vocab_size))
enc = CuDNNLSTM(hidden_dims, return_state=True)
_, state_h, state_c = enc(enc_inputs)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None, vocab_size))


AssertionError: Please provide to Input either a `shape` or a `batch_shape` argument. Note that `shape` does not include the batch dimension.

[6, 2003, 160, 1954, 167, 420, 5, 2003, 317, 160, 1954]

In [28]:
output = ""
for i in target_tokens[2]:
    output += tokenizer.IdToPiece(i)
    
output

'<こんにちは。>'

In [19]:
output = ""
for i in input_tokens[0]:
    output += tokenizer.IdToPiece(i)
    
output

'▁Go.'